<a href="https://colab.research.google.com/github/subhanghi1/CODECLAUSE/blob/main/LANGUAGEMODELTRAIN_AI4BHARAT_VITS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!sudo apt-get install espeak-ng
# core deps
!pip install numpy==1.21.6
!pip install cython==0.29.28
!pip install scipy>=1.4.0
!pip install torch>=1.7
!pip install torchaudio
!pip install soundfile
!pip install librosa==0.8.0
!pip install numba==0.55.1
!pip install inflect==5.6.0
!pip install tqdm
!pip install anyascii
!pip install pyyaml
!pip install fsspec>=2021.04.0
!pip install packaging
# deps for examples
!pip install flask
# deps for inference
!pip install pysbd
# deps for notebooks
!pip install umap-learn==0.5.1
!pip install pandas
# deps for training
!pip install matplotlib
# coqui stack
!pip install trainer==0.0.20
# config management
!pip install coqpit>=0.0.16
# chinese g2p deps
!pip install jieba
!pip install pypinyin
# japanese g2p deps
!pip install mecab-python3==1.0.5
!pip install unidic-lite==1.0.8
# gruut+supported langs
!pip install gruut[de]==2.2.3
# deps for korean
!pip install jamo
!pip install nltk
!pip install g2pkk>=0.1.1
!pip install TTS

In [ ]:
from google.colab import drive

drive.mount("/content/drive")

Mounted at /content/drive


In [ ]:
import os

ds_name = "vits-vctk-narrator-22k-ds" #@param {type:"string"}
output_directory = "traineroutput" #@param {type:"string"}
upload_dir = "vits-ml-sampleuploads" #@param {type:"string"}
MODEL_FILE = "/root/.local/share/tts/tts_models--en--ljspeech--vits/model_file.pth" #@param {type:"string"}
upload_dir = "/content/drive/MyDrive/" + upload_dir
RUN_NAME = "VITS-en" #@param {type:"string"}


OUT_PATH = "/content/drive/MyDrive/"+ds_name+"/traineroutput/"
!mkdir $upload_dir
!mkdir /content/drive/MyDrive/$ds_name
!mkdir /content/drive/MyDrive/$ds_name/wavs/

In [ ]:
run_type = "restore" #@param ["continue","restore","restore-ckpt","newmodel"]
print(run_type + " run selected")

In [ ]:
#@title
!pip install pyloudnorm
!git clone https://github.com/xiph/rnnoise.git
!sudo apt-get install curl autoconf automake libtool python-dev pkg-config sox ffmpeg
%cd /content/rnnoise
!sh autogen.sh
!sh configure
!make clean
!make

In [ ]:
#@title
%cd /content
!sudo apt install sox
!git clone https://github.com/openai/whisper.git
!pip install git+https://github.com/openai/whisper.git


In [ ]:
#@title
%cd /content
!sudo apt-get install espeak-ng
!git clone https://github.com/coqui-ai/TTS.git
%cd TTS
!pip install -e .[all,dev,notebooks]
!pip install TTS

/content
Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following package was automatically installed and is no longer required:
  libnvidia-common-510
Use 'sudo apt autoremove' to remove it.
The following additional packages will be installed:
  espeak-ng-data libespeak-ng1 libpcaudio0 libsonic0
The following NEW packages will be installed:
  espeak-ng espeak-ng-data libespeak-ng1 libpcaudio0 libsonic0
0 upgraded, 5 newly installed, 0 to remove and 21 not upgraded.
Need to get 4,215 kB of archives.
After this operation, 12.0 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu focal/main amd64 libpcaudio0 amd64 1.1-4 [7,908 B]
Get:2 http://archive.ubuntu.com/ubuntu focal/main amd64 libsonic0 amd64 0.2.0-8 [13.1 kB]
Get:3 http://archive.ubuntu.com/ubuntu focal/main amd64 espeak-ng-data amd64 1.50+dfsg-6 [3,682 kB]
Get:4 http://archive.ubuntu.com/ubuntu focal/main amd64 libespeak-ng1 amd64 1.50+dfsg-6 [189 

In [ ]:
#@title
%cd $upload_dir
!ls -al

In [ ]:
subfolder = "narrator" #@param {type:"string"}
newspeakername = "narrator" #@param {type:"string"}

In [ ]:
run_denoise = "True" #@param ["True", "False"]
run_splits = "True" #@param ["True", "False"]
use_audio_filter = "True" #@param ["True", "False"]
normalize_audio = "True" #@param ["True", "False"]
#start_sil_dur = 0.2 #@param {type:"number"}
#end_sil_dur = 0.2 #@param {type:"number"}
#sample_max = 8 #@param {type:"number"}


In [ ]:
#@title

from pathlib import Path
import os
import subprocess
import soundfile as sf
import pyloudnorm as pyln
import sys
import glob
%cd $upload_dir
%cd $subfolder

#Normalize file extensions, convert all to lower case. Use -n flag to test.
#https://stackoverflow.com/a/32729737
!rename 's/\.([^.]+)$/.\L$1/' *

#!ls -al
!rm -rf $upload_dir/$subfolder/22k_1ch
!mkdir $upload_dir/$subfolder/22k_1ch

!find . -name '*.mp3' -exec bash -c 'for f; do ffmpeg -hide_banner -loglevel error -i "$f" -acodec pcm_s16le -ar 22050 -ac 1 22k_1ch/"${f%.mp3}".wav ; done' _ {} +
!find . -name '*.ogg' -exec bash -c 'for f; do ffmpeg -hide_banner -loglevel error -i "$f" -acodec pcm_s16le -ar 22050 -ac 1 22k_1ch/"${f%.ogg}".wav ; done' _ {} +
!find . -name '*.wav' -exec bash -c 'for f; do ffmpeg -hide_banner -loglevel error -i "$f" -acodec pcm_s16le -ar 22050 -ac 1 22k_1ch/"${f%.wav}".wav ; done' _ {} +
!ls -al $upload_dir/$subfolder/22k_1ch
print("Files converted to 22khz 1ch wav")
%cd $upload_dir
%cd $subfolder
!rm temp.raw
!rm rnn.raw
%cd 22k_1ch
!find . -name "*.wav" -type f -size -35k -delete
%cd $upload_dir
#Convert and resample uploaded mp3/wav clips to 1 channel, 22khz

#
if run_denoise=="True":
  print("Running denoise...")
  orig_wavs= upload_dir + '/' + subfolder + "/22k_1ch/"
  print(orig_wavs)

  from pathlib import Path
  import os
  import subprocess
  import soundfile as sf
  import pyloudnorm as pyln
  import sys
  import glob
  rnn = "/content/rnnoise/examples/rnnoise_demo"
  paths = glob.glob(os.path.join(orig_wavs, '*.wav'))
  for filepath in paths:
    base = os.path.basename(filepath)
    tp_s = upload_dir + '/' + subfolder + "/22k_1ch/denoise/"
    tf_s = upload_dir + '/' + subfolder + "/22k_1ch/denoise/" + base
    target_path = Path(tp_s)
    target_file = Path(tf_s)
    print("From: " + str(filepath))
    print("To: " + str(target_file))



  # Stereo to Mono; upsample to 48000Hz
  # added -G to fix gain, -v 0.8
    subprocess.run(["sox", "-G", "-v", "0.8", filepath, "48k.wav", "remix", "-", "rate", "48000"])
    subprocess.run(["sox", "48k.wav", "-c", "1", "-r", "48000", "-b", "16", "-e", "signed-integer", "-t", "raw", "temp.raw"]) # convert wav to raw
    subprocess.run(["/content/rnnoise/examples/rnnoise_demo", "temp.raw", "rnn.raw"]) # apply rnnoise
    subprocess.run(["sox", "-G", "-v", "0.8", "-r", "48k", "-b", "16", "-e", "signed-integer", "rnn.raw", "-t", "wav", "rnn.wav"]) # convert raw back to wav

    subprocess.run(["mkdir", "-p", str(target_path)])
    if use_audio_filter=="True":
      print("Running highpass/lowpass & resample")
      subprocess.run(["sox", "rnn.wav", str(target_file), "remix", "-", "highpass", "50", "lowpass", "8000", "rate", "22050"])
      # apply high/low pass filter and change sr to 22050Hz
      data, rate = sf.read(target_file)
    elif use_audio_filter=="False":
      print("Running resample without filter")
      subprocess.run(["sox", "rnn.wav", str(target_file), "remix", "-", "rate", "22050"])
      # apply high/low pass filter and change sr to 22050Hz
      data, rate = sf.read(target_file)
# peak normalize audio to -6 dB
    if normalize_audio=="True":
      print("Output normalized")
      peak_normalized_audio = pyln.normalize.peak(data, -6.0)

# measure the loudness first
      meter = pyln.Meter(rate) # create BS.1770 meter
      loudness = meter.integrated_loudness(data)

# loudness normalize audio to -25 dB LUFS
      loudness_normalized_audio = pyln.normalize.loudness(data, loudness, -25.0)
      sf.write(target_file, data=loudness_normalized_audio, samplerate=22050)
      print("")
    elif normalize_audio=="False":
      print("File written without normalizing")
      sf.write(target_file, data=data, samplerate=22050)
      print("")

  !rm $target_path/rnn.wav
  !rm $target_path/48k.wav

elif run_denoise=="False":
  paths = glob.glob(os.path.join(orig_wavs, '*.wav'))
  for filepath in paths:
    print("Skipping denoise...")
    base = os.path.basename(filepath)
    tp_s = upload_dir + '/' + subfolder + "/22k_1ch/denoise/"
    tf_s = upload_dir + '/' + subfolder + "/22k_1ch/denoise/" + base
    target_path = Path(tp_s)
    target_file = Path(tf_s)
    print("From: " + str(filepath))
    print("To: " + str(target_file))
    subprocess.run(["sox", "-G", "-v", "0.8", filepath, "48k.wav", "remix", "-", "rate", "48000"])
    subprocess.run(["sox", "48k.wav", "-c", "1", "-r", "48000", "-b", "16", "-e", "signed-integer", "-t", "raw", "temp.raw"]) # convert wav to raw
    #subprocess.run(["/content/rnnoise/examples/rnnoise_demo", "temp.raw", "rnn.raw"]) # apply rnnoise
    subprocess.run(["sox", "-G", "-v", "0.8", "-r", "48k", "-b", "16", "-e", "signed-integer", "rnn.raw", "-t", "wav", "rnn.wav"]) # convert raw back to wav
    subprocess.run(["mkdir", "-p", str(target_path)])
    if use_audio_filter=="True":
      print("Running filter...")
      subprocess.run(["sox", "rnn.wav", str(target_file), "remix", "-", "highpass", "50", "lowpass", "8000", "rate", "22050"]) # apply high/low pass filter and change sr to 22050Hz
      data, rate = sf.read(target_file)
    elif use_audio_filter=="False":
      print("Skipping filter...")
      subprocess.run(["sox", "rnn.wav", str(target_file), "remix", "-", "rate", "22050"]) # apply high/low pass filter and change sr to 22050Hz
      data, rate = sf.read(target_file)
          # peak normalize audio to -6 dB
    if normalize_audio=="True":
      print("Output normalized")
      peak_normalized_audio = pyln.normalize.peak(data, -6.0)

# measure the loudness first
      meter = pyln.Meter(rate) # create BS.1770 meter
      loudness = meter.integrated_loudness(data)

# loudness normalize audio to -25 dB LUFS
      loudness_normalized_audio = pyln.normalize.loudness(data, loudness, -25.0)
      sf.write(target_file, data=loudness_normalized_audio, samplerate=22050)
      print("")
    if normalize_audio=="False":
      print("File written without normalizing")
      sf.write(target_file, data=data, samplerate=22050)
      print("")
  !rm $target_path/rnn.wav
  !rm $target_path/48k.wav

if run_splits=="False":
  print("Copying files without splitting...")
  %mkdir /content/drive/MyDrive/$ds_name
  %mkdir /content/drive/MyDrive/$ds_name/wav48_silence_trimmed
  %mkdir /content/drive/MyDrive/$ds_name/wav48_silence_trimmed/$newspeakername

  !cp $target_path/*.wav /content/drive/MyDrive/$ds_name/wav48_silence_trimmed/$newspeakername
if run_splits=="True":
  %mkdir /content/drive/MyDrive/$ds_name
  %mkdir /content/drive/MyDrive/$ds_name/wav48_silence_trimmed
  %mkdir /content/drive/MyDrive/$ds_name/wav48_silence_trimmed/$newspeakername
  print("Splitting output and copying...")
  %cd $target_path
  !rm -rf splits
  !mkdir splits
  !for FILE in *.wav; do sox "$FILE" splits/"$FILE" --show-progress silence 1 0.2 0.1% 1 0.2 0.1% : newfile : restart ; done
#alt split method: force splits of 9.5 seconds, however this will split words. Comment the above with # and remove the # below to change
#!for FILE in *.wav; do sox "$FILE" splits/"$FILE" --show-progress trim 0 8 : restart ; done
  %cd splits
  !mkdir resplit
  !for FILE in *.wav; do sox "$FILE" resplit/"$FILE" --show-progress trim 0 9 : newfile : restart ; done
  %cd resplit
  !find . -name "*.wav" -type f -size -35k -delete
  #!ls -al
  %cd /content/drive/MyDrive/$ds_name/wav48_silence_trimmed/$newspeakername

#  !ls -al
!cp $target_path/splits/resplit/*.wav /content/drive/MyDrive/$ds_name/wav48_silence_trimmed/$newspeakername
%cd /content/drive/MyDrive/$ds_name/wav48_silence_trimmed/$newspeakername
!rm *.flac
!find . -name '*.wav' -exec bash -c 'for f; do ffmpeg -i "$f" -c:a flac "${f%.wav}"_mic1.flac ; done' _ {} +
!rm *.wav
!ls /content/drive/MyDrive/$ds_name/wav48_silence_trimmed/$newspeakername

In [ ]:
whisper_model = "large-v2" #@param ["large-v2", "large-v1", "medium.en", "small.en", "base.en"]


In [ ]:
#@title
import whisper
import os, os.path
import glob
import pandas as pd

from pathlib import Path


#model = whisper.load_model("medium.en")
model = whisper.load_model(whisper_model)

In [ ]:
#@title
!ls /content/drive/MyDrive/$ds_name/wav48_silence_trimmed

In [ ]:
newspeakername = "narrator" #@param {type:"string"}


In [ ]:
from transformers import WhisperProcessor, WhisperForConditionalGeneration
options = dict(language="english", beam_size=5, best_of=5)
transcribe_options = dict(task="transcribe", **options)

#@title
wavs = '/content/drive/MyDrive/'+ds_name+'/wav48_silence_trimmed/'+newspeakername


"""
if os.path.exists(modelpath):
	if os.path.isfile(modelpath+"large-v2.pt"):
		print("Saved large-v2 found")
		model = whisper.load_model(download_root=modelpath,name="large-v2")
else:
	print("loading model from Huggingface")
	model = whisper.load_model("large-v2")
"""
paths = glob.glob(os.path.join(wavs, '*.flac'))
print(len(paths))
all_filenames = []
transcript_text = []
try:
	os.mkdir('/content/drive/MyDrive/'+ds_name+'/txt/')
except OSError as error:
	print(error)
try:
	os.mkdir('/content/drive/MyDrive/'+ds_name+'/txt/'+newspeakername+'/')
except OSError as error:
	print(error)

for filepath in paths:
	base = os.path.basename(filepath)
	all_filenames.append(base)
	result = model.transcribe(filepath)
	output = result["text"].lstrip()
	output = output.replace("\n","")
	print(output)
	thefile = str(os.path.basename(filepath).lstrip(".")).rsplit(".")[0]
	thefile = thefile[:-5]
	print(thefile)
	outfilepath = '/content/drive/MyDrive/'+ds_name+'/txt/'+newspeakername+'/'+thefile+'.txt'
	with open(outfilepath, 'w', encoding='utf-8') as indfile:
		indfile.write(output)

In [ ]:
ds_name2 = "vits-vctk-ms-en-ds" #@param {type:"string"}
newspeakername2 = "narrator" #@param {type:"string"}

In [ ]:
#@title
import os, os.path
import glob
import shutil

flac = '/content/drive/MyDrive/'+ds_name2+'/wav48_silence_trimmed/'+newspeakername2+'/'
wav = '/content/drive/MyDrive/'+ds_name2+'/wavs/'
txt = '/content/drive/MyDrive/'+ds_name2+'/txt/'+newspeakername2+'/'
outfilepath = '/content/drive/MyDrive/'+ds_name2+'/txt/'+newspeakername2
backup_path = '/content/drive/MyDrive/'+ds_name2+'/badfiles/'
if not os.path.exists(backup_path):
    os.makedirs(backup_path)
searchstr = "(?i)(?!(?![×Þß÷þø])[a-zÀ-ÿ])\S"
txtfiles = glob.glob(os.path.join(txt, '*.txt'))
EMPTY_TRANSCRIPTS = []
for txts in txtfiles:
    with open(txts, 'r', encoding='utf-8') as outfile:

      state = os.stat(txts).st_size == 0
      if state==True:
          print(str(txts))
          basename_without_ext = os.path.splitext(os.path.basename(txts))[0]
          print(basename_without_ext)
          bad_wav = basename_without_ext+".wav"
          bad_txt = basename_without_ext+".txt"
          bad_flac = basename_without_ext+"_mic1.flac"
          print(bad_wav)
          print(bad_txt)
          bad_wav_file_size = os.path.getsize(wav+bad_wav)
          bad_txt_file_size = os.path.getsize(txt+bad_txt)
          bad_flac_file_size = os.path.getsize(flac+bad_flac)
          print(bad_wav_file_size)
          print(bad_txt_file_size)
          shutil.move(wav+bad_wav, backup_path+bad_wav)
          shutil.move(txt+bad_txt, backup_path+bad_txt)
          shutil.move(flac+bad_flac, backup_path+bad_flac)





In [ ]:
#@title
!tts --text "I am the very model of a modern Major General" --model_name "tts_models/en/ljspeech/vits" --out_path /content/ljspeech-vits.wav
#!tts --text "I am the very model of a modern Major General" --model_name "tts_models/en/vctk/vits" --out_path /content/ljspeech-vits.wav

 > Downloading model to /root/.local/share/tts/tts_models--en--ljspeech--vits
100% 146M/146M [00:16<00:00, 8.86MiB/s]
 > Model's license - apache 2.0
 > Check https://choosealicense.com/licenses/apache-2.0/ for more info.
 > Using model: vits
 > Setting up Audio Processor...
 | > sample_rate:22050
 | > resample:False
 | > num_mels:80
 | > log_func:np.log10
 | > min_level_db:0
 | > frame_shift_ms:None
 | > frame_length_ms:None
 | > ref_level_db:None
 | > fft_size:1024
 | > power:None
 | > preemphasis:0.0
 | > griffin_lim_iters:None
 | > signal_norm:None
 | > symmetric_norm:None
 | > mel_fmin:0
 | > mel_fmax:None
 | > pitch_fmin:None
 | > pitch_fmax:None
 | > spec_gain:20.0
 | > stft_pad_mode:reflect
 | > max_norm:1.0
 | > clip_norm:True
 | > do_trim_silence:False
 | > trim_db:60
 | > do_sound_norm:False
 | > do_amp_to_db_linear:True
 | > do_amp_to_db_mel:True
 | > do_rms_norm:False
 | > db_level:None
 | > stats_path:None
 | > base:10
 | > hop_length:256
 | > win_length:1024
 > Text: I a

In [ ]:
!tts --list_models

In [ ]:
import torch
%load_ext tensorboard

In [ ]:
%tensorboard --logdir /content/drive/MyDrive/$ds_name/$output_directory/

In [ ]:
#@title
!ls -al /content/drive/MyDrive/$ds_name/traineroutput

In [ ]:
run_folder = "vits_vctk-February-09-2023_12+12AM-914280a5" #@param {type:"string"}


In [ ]:
#@title
!ls -al /content/drive/MyDrive/$ds_name/traineroutput/$run_folder

In [ ]:
ckpt_file = "best_model_1003928.pth" #@param {type:"string"}
print(ckpt_file + " selected for restore run")
if run_type=="continue":
  print("Warning:\n restore checkpoint selected, but run type set to continue.\nTrainer will load best loss from checkpoint directory.\n Are you sure this is what you want to do?\n\nIf not, change the run type below to 'restore'")
elif run_type=="restore-ckpt":
  print("Warning:\n restore checkpoint selected, run type set to restore from selected checkpoint, not default base model.\nIf this is not correct, adjust the run type.")


In [ ]:
run_type = "restore" #@param ["continue","restore","restore-ckpt","newmodel"]
print(run_type + " run selected")

In [ ]:
import os

from trainer import Trainer, TrainerArgs

from TTS.tts.configs.shared_configs import BaseDatasetConfig
from TTS.tts.configs.vits_config import VitsConfig
from TTS.tts.datasets import load_tts_samples
from TTS.tts.models.vits import Vits, VitsArgs, VitsAudioConfig
from TTS.tts.utils.speakers import SpeakerManager
from TTS.tts.utils.text.tokenizer import TTSTokenizer
from TTS.utils.audio import AudioProcessor
from TTS.bin.compute_embeddings import compute_embeddings
from TTS.tts.utils.data import get_length_balancer_weights
from TTS.tts.utils.languages import LanguageManager, get_language_balancer_weights
from TTS.tts.utils.speakers import SpeakerManager, get_speaker_balancer_weights, get_speaker_manager

In [ ]:
  #output_path = os.path.dirname(os.path.abspath(__file__))
output_path = os.path.dirname("/content/drive/MyDrive/"+ds_name+"/traineroutput/")
print(output_path)
SKIP_TRAIN_EPOCH=False
#https://github.com/coqui-ai/TTS/releases/tag/speaker_encoder_model
## Extract speaker embeddings

SPEAKER_ENCODER_CHECKPOINT_PATH = (
    "https://github.com/coqui-ai/TTS/releases/download/speaker_encoder_model/model_se.pth.tar"
)

SPEAKER_ENCODER_CONFIG_PATH = "https://github.com/coqui-ai/TTS/releases/download/speaker_encoder_model/config_se.json"
SKIP_TRAIN_EPOCH = False
BATCH_SIZE = 16
SAMPLE_RATE = 22050
MAX_AUDIO_LEN_IN_SECONDS = 10
NUM_RESAMPLE_THREADS = 10

#Add additional datasets:
#dataset_var_name_here = BaseDatasetConfig(
#    formatter="vctk", meta_file_train="", phonemizer="espeak", dataset_name="dataset_name", language="en-us", path="/content/drive/MyDrive/"+path_to_dataset
#)

dataset_config = BaseDatasetConfig(
    formatter="vctk", meta_file_train="", phonemizer="espeak", dataset_name="vctk-vits", language="en-us", path="/content/drive/MyDrive/"+ds_name
)

#Add additional datasets:
#DATASETS_CONFIG_LIST = [dataset_config,dataset_var_name_here,etc...]
#

DATASETS_CONFIG_LIST = [dataset_config]
D_VECTOR_FILES=[]

In [ ]:
audio_config = VitsAudioConfig(
    sample_rate=22050, win_length=1024, hop_length=256, num_mels=80, mel_fmin=0, mel_fmax=None
)

vitsArgs = VitsArgs(
    use_d_vector_file=True,
    d_vector_file=D_VECTOR_FILES,
    d_vector_dim=512,
    num_layers_text_encoder=6,
    embedded_language_dim=4,
    speaker_encoder_model_path=SPEAKER_ENCODER_CHECKPOINT_PATH,
    speaker_encoder_config_path=SPEAKER_ENCODER_CONFIG_PATH,
    use_language_embedding=False,
    use_speaker_embedding=False,
    use_speaker_encoder_as_loss=True,
    use_sdp=True,
)

config = VitsConfig(
    #https://github.com/coqui-ai/TTS/pull/2234#issuecomment-1369538965
    #max_text_len=325,  # change this if you have a larger VRAM than 16GB
    #use_weighted_sampler=False,
    audio=audio_config,
    batch_group_size=16,
    batch_size=16,
    compute_input_seq_cache=True,
    cudnn_benchmark=False,
    datasets=[dataset_config],
    epochs=10000,
    eval_batch_size=16,
    eval_split_max_size=256,
    log_model_step=100,
    max_audio_len=SAMPLE_RATE * MAX_AUDIO_LEN_IN_SECONDS,
    max_text_len=325,
    min_audio_len=1,
    min_text_len=1,
    mixed_precision=False,
    model_args=vitsArgs,
    num_eval_loader_workers=2,
    num_loader_workers=2,
    output_path=output_path,
    phoneme_cache_path=os.path.join(output_path, "phoneme_cache"),
    phoneme_language="en-us",
    print_eval=True,
    print_step=50,
    run_eval=True,
    run_name="vits_vctk",
    save_best_after=1000,
    save_checkpoints=True,
    save_n_checkpoints=4,
    save_step=1000,
    speaker_encoder_loss_alpha=9.0,
    start_by_longest=True,
    test_delay_epochs=-1,
    text_cleaner="multilingual_cleaners",
    use_phonemes=True,
    use_weighted_sampler=False,
    weighted_sampler_attrs={"speaker_name": 1.0},
    weighted_sampler_multipliers={"speaker_name": {}},
#    test_sentences=[
#        [
#            "It took me quite a long time to develop a voice, and now that I have it I'm not going to be silent.",
#            "VCTK_ah1",
#            None,
#            "en",
#        ],
#        [
#            "Be a voice, not an echo.",
#            "VCTK_ah1",
#            None,
#            "en",
#        ],
#        [
#            "I'm sorry Dave. I'm afraid I can't do that.",
#            "VCTK_rs1",
#            None,
#            "en",
#        ],
#        [
#            "This cake is great. It's so delicious and moist.",
#            "VCTK_rs1",
#            None,
#            "en",
#        ],
#    ]
)

In [ ]:
# INITIALIZE THE AUDIO PROCESSOR
# Audio processor is used for feature extraction and audio I/O.
# It mainly serves to the dataloader and the training loggers.
ap = AudioProcessor.init_from_config(config)

In [ ]:
for dataset_conf in DATASETS_CONFIG_LIST:
    # Check if the embeddings weren't already computed, if not compute it
    print(dataset_conf.path)
    embbase=str(dataset_conf.dataset_name)
    #embeddings_file = MODEL_DIR+"speakers.pth"
    embeddings_file = os.path.join(dataset_conf.path, embbase+"_speakers.pth")
    print(embeddings_file)
    if not os.path.isfile(embeddings_file):
        print(f">>> Computing the speaker embeddings for the {dataset_conf.dataset_name} dataset")
        print(SPEAKER_ENCODER_CHECKPOINT_PATH)
        print(SPEAKER_ENCODER_CONFIG_PATH)
        print(embeddings_file)
        print(dataset_conf.formatter)
        print(dataset_conf.dataset_name)
        print(dataset_conf.path)
        print(dataset_conf.meta_file_train)
        print(dataset_conf.meta_file_val)
        compute_embeddings(
            SPEAKER_ENCODER_CHECKPOINT_PATH,
            SPEAKER_ENCODER_CONFIG_PATH,
            embeddings_file,
            old_spakers_file=None,
            config_dataset_path=None,
            formatter_name=dataset_conf.formatter,
            dataset_name=dataset_conf.dataset_name,
            dataset_path=dataset_conf.path,
            meta_file_train=dataset_conf.meta_file_train,
            meta_file_val=dataset_conf.meta_file_val,
            disable_cuda=False,
            no_eval=False,
        )
    D_VECTOR_FILES.append(embeddings_file)

In [ ]:

speaker_manager = SpeakerManager(
    d_vectors_file_path=D_VECTOR_FILES,
    encoder_model_path=SPEAKER_ENCODER_CHECKPOINT_PATH,
    encoder_config_path=SPEAKER_ENCODER_CONFIG_PATH)

ALL_SPEAKERS = []
ALL_SPEAKERS = speaker_manager.speaker_names
ALL_SENTENCES = []
TEST_SENTENCES_PH1 = "I am the very model of a modern Major-General, I've information vegetable, animal, and mineral,  I know the kings of England, and I quote the fights historical. From Marathon to Waterloo, in order categorical"
TEST_SENTENCES_PH2 = "I'm sorry, but I don’t want to be an emperor. That's not my business. I don't want to rule or conquer anyone. I should like to help everyone, if possible; Jew, Gentile, black man, white. We all want to help one another. Human beings are like that. We want to live by each other's happiness - not by each other's misery. We don't want to hate and despise one another. In this world there is room for everyone. And the good earth is rich and can provide for everyone. The way of life can be free and beautiful, but we have lost the way."

print(ALL_SPEAKERS)
for speakername in ALL_SPEAKERS:
    ph1 = [TEST_SENTENCES_PH1,speakername,None,"en-us"]
    ph2 = [TEST_SENTENCES_PH2,speakername,None,"en-us"]
    ALL_SENTENCES.append(ph1)
    ALL_SENTENCES.append(ph2)
#print(ALL_SENTENCES)
config.test_sentences=ALL_SENTENCES

In [ ]:
tokenizer, config = TTSTokenizer.init_from_config(config)


In [ ]:
model = Vits(config, ap, tokenizer, speaker_manager)

 > External Speaker Encoder Loaded !!


In [ ]:
train_samples, eval_samples = load_tts_samples(
    DATASETS_CONFIG_LIST,
    eval_split=True,
    eval_split_max_size=config.eval_split_max_size,
    eval_split_size=config.eval_split_size,
)


In [ ]:
#@title
import os
import re

from trainer import Trainer, TrainerArgs

from TTS.tts.configs.shared_configs import BaseDatasetConfig
from TTS.tts.configs.vits_config import VitsConfig
from TTS.tts.datasets import load_tts_samples
from TTS.tts.models.vits import Vits, VitsArgs, VitsAudioConfig
from TTS.tts.utils.speakers import SpeakerManager
from TTS.tts.utils.text.tokenizer import TTSTokenizer
from TTS.utils.audio import AudioProcessor
from TTS.bin.compute_embeddings import compute_embeddings
from TTS.tts.utils.data import get_length_balancer_weights
from TTS.tts.utils.languages import LanguageManager, get_language_balancer_weights
from TTS.tts.utils.speakers import SpeakerManager, get_speaker_balancer_weights, get_speaker_manager

from tqdm.contrib.concurrent import process_map

from TTS.config import load_config
from TTS.tts.datasets import load_tts_samples
from TTS.tts.utils.text.phonemizers import espeak_wrapper
from TTS.tts.utils.text.phonemizers import ESpeak
import multiprocessing

def compute_phonemes(item):
    text = item["text"]
    ph = phonemizer.phonemize(text).replace("|", "")
    return set(list(ph))
train_samples, eval_samples = load_tts_samples(
    DATASETS_CONFIG_LIST,
    eval_split=True,
    eval_split_max_size=config.eval_split_max_size,
    eval_split_size=config.eval_split_size,
)

#from TTS/bin/find_unique_chars.py
items = train_samples + eval_samples

texts = "".join(item["text"] for item in items)
chars = set(texts)
lower_chars = filter(lambda c: c.islower(), chars)
chars_force_lower = [c.lower() for c in chars]
chars_force_lower = set(chars_force_lower)

print(f" > Number of unique characters: {len(chars)}")
print(f" > Unique characters: {''.join(sorted(chars))}")
print(f" > Unique lower characters: {''.join(sorted(lower_chars))}")
print(f" > Unique all forced to lower characters: {''.join(sorted(chars_force_lower))}")
#https://www.geeksforgeeks.org/python-convert-string-to-unicode-characters/
char_str = str(chars)


 | > Found 766 files in /content/drive/MyDrive/vits-vctk-narrator-22k-ds
 > Number of unique characters: 66
 > Unique characters:  %',-.012345678?ABCDEFGHIJKLMNOPQRSTUVWYabcdefghijklmnopqrstuvwxyz
 > Unique lower characters: abcdefghijklmnopqrstuvwxyz
 > Unique all forced to lower characters:  %',-.012345678?abcdefghijklmnopqrstuvwxyz


In [ ]:
print("Current reinit_text_encoder value: " + str(config.model_args.reinit_text_encoder))
reinit_te_status = "False" #@param ["False", "True"]
if reinit_te_status=="False":
  print("Text encoder will not be reinitilized")
elif reinit_te_status=="True":
  config.model_args.reinit_text_encoder=True
  print("Model arguments set to reinitilize text encoder")
  print("Current reinit_DP value: " + str(config.model_args.reinit_DP))
reinit_DP_status = "False" #@param ["False", "True"]
if reinit_DP_status=="False":
  print("DP will not be reinitilized")
elif reinit_DP_status=="True":
  config.model_args.reinit_DP=True
  print("Model arguments set to reinitilize DP")
print("Current freeze_waveform_decoder value: " + str(config.model_args.freeze_waveform_decoder))
freeze_waveform_decoder_status = "False" #@param ["False", "True"]
if freeze_waveform_decoder_status=="False":
  print("Waveform decoder will NOT be frozen")
  config.model_args.freeze_waveform_decoder=False
elif freeze_waveform_decoder_status=="True":
  config.model_args.freeze_waveform_decoder=True
  print("Waveform decoder FROZEN")
print("Current freeze_flow_decoder value: " + str(config.model_args.freeze_flow_decoder))
freeze_flow_decoder_status = "False" #@param ["False", "True"]
if freeze_flow_decoder_status=="False":
  print("Flow decoder will NOT be frozen")
  config.model_args.freeze_flow_decoder=None
elif freeze_flow_decoder_status=="True":
  config.model_args.freeze_flow_decoder="True"
  print("Flow decoder FROZEN")
print("Current freeze_encoder value: " + str(config.model_args.freeze_encoder))
freeze_encoder_status = "False" #@param ["False", "True"]
if freeze_encoder_status=="False":
  print("Text encoder will NOT be frozen")
  config.model_args.freeze_encoder=False
elif freeze_encoder_status=="True":
  config.model_args.freeze_encoder=True
  print("Text encoder FROZEN")
print("Current freeze_DP value: " + str(config.model_args.freeze_DP))
freeze_DP_status = "False" #@param ["False", "True"]
if freeze_DP_status=="False":
  print("Duration predictor will NOT be frozen")
  config.model_args.freeze_DP=False
elif freeze_DP_status=="True":
  config.model_args.freeze_DP=True
  print("Duration predictor FROZEN")

In [ ]:
#@title
print(run_type)

if run_type=="continue":
  CONTINUE_PATH="/content/drive/MyDrive/"+ds_name+"/traineroutput/"+run_folder
  trainer = Trainer(
    TrainerArgs(continue_path=CONTINUE_PATH, skip_train_epoch=SKIP_TRAIN_EPOCH),
    config,
    output_path=OUT_PATH,
    model=model,
    train_samples=train_samples,
    eval_samples=eval_samples,
)
elif run_type=="restore":
    trainer = Trainer(
    TrainerArgs(restore_path=MODEL_FILE, skip_train_epoch=SKIP_TRAIN_EPOCH),
    config,
    output_path=OUT_PATH,
    model=model,
    train_samples=train_samples,
    eval_samples=eval_samples,
)
elif run_type=="restore-ckpt":
  trainer = Trainer(
  TrainerArgs(restore_path="/content/drive/MyDrive/"+ds_name+"/traineroutput/"+run_folder+"/"+ckpt_file, skip_train_epoch=SKIP_TRAIN_EPOCH),
  config,
  output_path=OUT_PATH,
  model=model,
  train_samples=train_samples,
  eval_samples=eval_samples,
)
elif run_type=="newmodel":
  trainer = Trainer(
  TrainerArgs(),
  config,
  output_path=OUT_PATH,
  model=model,
  train_samples=train_samples,
  eval_samples=eval_samples,
)

In [ ]:
trainer.fit()

In [ ]:
!tts --model_path /content/drive/MyDrive/vits-vctk-multi-ds/traineroutput/vits_vctk-February-09-2023_12+12AM-914280a5/best_model_1003928.pth \
--config_path /content/drive/MyDrive/vits-vctk-multi-ds/traineroutput/vits_vctk-February-09-2023_12+12AM-914280a5/config.json \
--list_speaker_idxs \
--text ""

In [ ]:
out_wav_file ="/content/drive/MyDrive/me-mmj.wav"

In [ ]:
!tts --model_path /content/drive/MyDrive/vits-vctk-multi-ds/traineroutput/vits_vctk-February-09-2023_12+12AM-914280a5/best_model_1003928.pth \
--config_path /content/drive/MyDrive/vits-vctk-multi-ds/traineroutput/vits_vctk-February-09-2023_12+12AM-914280a5/config.json \
--speaker_idx VCTK_me \
--text "I am the very model of a modern Major-General,\
 I've information vegetable, animal, and mineral, \
 I know the kings of England, and I quote the fights historical \
 From Marathon to Waterloo, in order categorical; \
 I'm very well acquainted, too, with matters mathematical, \
 I understand equations, both the simple and quadratical, \
  About binomial theorem I'm teeming with a lot o' news, \
  With many cheerful facts about the square of the hypotenuse." \
  --out_path $out_wav_file

 > Using model: vits
 > Setting up Audio Processor...
 | > sample_rate:22050
 | > resample:False
 | > num_mels:80
 | > log_func:np.log10
 | > min_level_db:0
 | > frame_shift_ms:None
 | > frame_length_ms:None
 | > ref_level_db:None
 | > fft_size:1024
 | > power:None
 | > preemphasis:0.0
 | > griffin_lim_iters:None
 | > signal_norm:None
 | > symmetric_norm:None
 | > mel_fmin:0
 | > mel_fmax:None
 | > pitch_fmin:None
 | > pitch_fmax:None
 | > spec_gain:20.0
 | > stft_pad_mode:reflect
 | > max_norm:1.0
 | > clip_norm:True
 | > do_trim_silence:False
 | > trim_db:60
 | > do_sound_norm:False
 | > do_amp_to_db_linear:True
 | > do_amp_to_db_mel:True
 | > do_rms_norm:False
 | > db_level:None
 | > stats_path:None
 | > base:10
 | > hop_length:256
 | > win_length:1024
 > Model fully restored. 
 > Setting up Audio Processor...
 | > sample_rate:16000
 | > resample:False
 | > num_mels:64
 | > log_func:np.log10
 | > min_level_db:-100
 | > frame_shift_ms:None
 | > frame_length_ms:None
 | > ref_level_db:

In [ ]:
from IPython.display import Audio
from IPython.display import display
wn = Audio(out_wav_file, autoplay=False) ##
display(wn)##